In [ ]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import numpy  as np
import pandas as pd

import root_numpy as rn

In [ ]:
#For use with output of OpFlash FlatTreeWriter
data = rn.root2rec('./OpMichel/flat_flash.root','_flash_tree')
flash_df = pd.DataFrame(data=data)

In [ ]:
flash_df = flash_df.set_index('_evt')
flash_df.drop('_flashnum',axis=1,inplace=True)

In [ ]:
flash_df['_TotalPE'].size

In [ ]:
highPE_df = flash_df.query('_TotalPE > 10')
highPE_df['_Time'].size

In [ ]:
#g returns all two point combintations
import itertools 
g = lambda x : itertools.combinations(x,2)

In [ ]:
#Does it work?
list(g(np.array([1,2,3])))

In [ ]:
#Return time variable from dataframe
ttime = lambda x, y, z : x.ix[y].iloc[z]['_Time']

#Return distance btweeen two flash points
def dist(df,y,z1,z2):
    y1 = df.ix[y].iloc[z1]['_YCenter']
    y2 = df.ix[y].iloc[z2]['_YCenter']
    z1 = df.ix[y].iloc[z1]['_ZCenter']
    z2 = df.ix[y].iloc[z2]['_ZCenter']
    
    return np.sqrt(np.power(y1-y2,2) + np.power(z1-z2,2))

In [ ]:
#The following method:
# *loops over events
# *checks if there is more than 1 flash in event
# *if so, compute spatial separation between flashes, and time difference
# *add time differences to td
# *add spatial differences to sd

td = {}
sd = {}

for z in highPE_df.index.unique():
    print "Looking at event...",z
    if highPE_df.ix[z]["_Time"].size < 2:
        print "Bad... not enough good flashes"; continue;
    td[z] = []
    sd[z] = []
    
    for c in g(np.arange(0,highPE_df.ix[z].index.values.size,1)):
        td[z].append((np.abs(ttime(highPE_df,z,c[1]) - ttime(highPE_df,z,c[0]))))
        sd[z].append(dist(highPE_df,z,c[0],c[1]))
    td[z] = np.array(td[z])
    sd[z] = np.array(sd[z])

In [ ]:
f1 = pd.DataFrame(sd.items(),columns=['_evt','sdiff']);
f1.set_index('_evt',inplace=True);

f2 = pd.DataFrame(td.items(),columns=['_evt','tdiff']);
f2.set_index('_evt',inplace=True);

In [ ]:
#plt.figure(figsize=(8,8))
#plt.hist2d(np.hstack(f2['tdiff'].values.flat),np.hstack(f1['sdiff'].values.flat),bins=100)
#plt.show()

#Histogram things
plt.figure(figsize=(8,8))
one = plt.hist(np.hstack(f2['tdiff'].values.flat) * 0.5,bins=30,range=(0,15))
plt.yscale('log', nonposy='clip')
plt.show()

plt.figure(figsize=(8,8))
plt.hist(np.hstack(f1['sdiff'].values.flat),bins=100,range=(0,1000))
plt.show()



In [ ]:
from scipy.stats import expon

In [ ]:
expon.fit(np.hstack(f2['tdiff'].values.flat) * 0.5)

In [ ]:
plt.plot(one[1][:one[1].size-1],one[0],"ro")
plt.yscale('log', nonposy='clip')

In [ ]:
one[0].sum()